<h1>Data has been Cleaned and Mered into a single dataset in this notebook</h1>

<h3>Loading the libraries</h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from io import StringIO
import re

<h3>Reading the Previous loan holding file</h3>

In [2]:
pre_loan= pd.read_csv("ModelBuild_Previous_Loan_Holdings.csv",encoding='latin1')

In [3]:
pre_loan.head()

,Client ID,Held Loan previously
0,1,1
1,2,0
2,3,0
3,4,1
4,5,0


In [4]:
pre_loan.shape

(10005, 2)

In [5]:
pre_loan.dtypes

Client ID                int64
Held Loan previously    object
dtype: object

In [6]:
#Checking for duplicates in the dataframe
pre_loan[pre_loan["Client ID"].duplicated()]

,Client ID,Held Loan previously
21,21,0
47,46,0
1222,1220,0
3678,3674,0
3679,3675,1


In [7]:
#Noting the index of the duplicate rows
i = pre_loan[pre_loan["Client ID"].duplicated()].index

In [8]:
#Removing the duplicate rows
pre_loan.drop(i, inplace = True)

In [9]:
pre_loan.shape

(10000, 2)

In [10]:
pre_loan.describe()

,Client ID
count,10000.00000
mean,5000.50000
std,2886.89568
min,1.00000
25%,2500.75000
50%,5000.50000
75%,7500.25000
max,10000.00000


In [11]:
#Checking for the types of values in the column
pre_loan["Held Loan previously"].unique()

array(['1', '0', '³', ' \x80-   '], dtype=object)

In [12]:
#Converting the values to 1 and 0 , Hence removing the other values from the column
int_list = []
for value in pre_loan["Held Loan previously"]:
    if value == '1':
        temp = 1
    elif value == '0':
        temp =0
    else:
        temp = 0
    int_list.append(temp)

In [13]:
#Assigning the new vlaues to the column
pre_loan["Held Loan previously"] = int_list

In [14]:
#Only 0 and 1 are are the values now 
pre_loan["Held Loan previously"].unique()

array([1, 0])

In [15]:
#Checking for null values 
pre_loan.isnull().sum()

Client ID               0
Held Loan previously    0
dtype: int64

In [16]:
#Number of clients that had taken the loan previously versus not taken the loan previously
pre_loan.groupby('Held Loan previously')['Client ID'].nunique()

Held Loan previously
0    7502
1    2498
Name: Client ID, dtype: int64

In [17]:
pre_loan.describe()

,Client ID,Held Loan previously
count,10000.00000,10000.000000
mean,5000.50000,0.249800
std,2886.89568,0.432919
min,1.00000,0.000000
25%,2500.75000,0.000000
50%,5000.50000,0.000000
75%,7500.25000,0.000000
max,10000.00000,1.000000


<h3> Reading the Demographics file</h3>

In [18]:
demographics = pd.read_csv("ModelBuild_Demographics.csv",encoding='latin1')

In [19]:
demographics.head()

,Client ID,Age,"Gender 1: Female, 2: Male",County,Income Group,Unnamed: 5
0,1,36,1,Cork,10001 - 40000,NaN
1,2,43,1,Cavan,0 - 10000,NaN
2,3,32,0,Dublin,10001 - 40000,NaN
3,4,52,1,Louth,40001 - 60000,NaN
4,5,63,0,Kilkenny,60001 - 100000,NaN


In [20]:
demographics.dtypes

Client ID                        int64
Age                              int64
Gender \n1: Female, 2: Male     object
County                          object
Income Group                    object
Unnamed: 5                     float64
dtype: object

In [21]:
#Removing the unwanted columns that were read
demographics.drop("Unnamed: 5", axis =1, inplace = True)

In [22]:
#Removing the speacial charecters from the column names
demographics.columns=demographics.columns.str.replace(':','')
demographics.columns=demographics.columns.str.replace(',','')
demographics.columns=demographics.columns.str.replace('\n','')

In [23]:
demographics.head()

,Client ID,Age,Gender 1 Female 2 Male,County,Income Group
0,1,36,1,Cork,10001 - 40000
1,2,43,1,Cavan,0 - 10000
2,3,32,0,Dublin,10001 - 40000
3,4,52,1,Louth,40001 - 60000
4,5,63,0,Kilkenny,60001 - 100000


In [24]:
#Changing the column name for more convinience
demographics.rename(columns = {"Gender 1 Female 2 Male": "Gender"}, inplace = True)

In [25]:
demographics.shape

(10004, 5)

In [26]:
#Checking for duplicates
demographics[demographics["Client ID"].duplicated()]

,Client ID,Age,Gender,County,Income Group
46,46,20,0,Laois,10001 - 40000
1221,1220,48,1,Kildare,100000+
3676,3674,37,1,Carlow,10001 - 40000
3678,3675,23,1,Cork,10001 - 40000


In [27]:
#Index of the duplicate rows
i = demographics[demographics["Client ID"].duplicated()].index

In [28]:
#Removing the duplicate rows
demographics.drop(i, inplace = True)

In [29]:
demographics.shape

(10000, 5)

In [30]:
#Checking for Null values
demographics.isnull().sum()

Client ID       0
Age             0
Gender          0
County          3
Income Group    0
dtype: int64

In [31]:
demographics.head()

,Client ID,Age,Gender,County,Income Group
0,1,36,1,Cork,10001 - 40000
1,2,43,1,Cavan,0 - 10000
2,3,32,0,Dublin,10001 - 40000
3,4,52,1,Louth,40001 - 60000
4,5,63,0,Kilkenny,60001 - 100000


In [32]:
#Checking for unique values in the gender column
demographics["Gender"]. unique()

array(['1', '0', 'M', 'Male', 'f', 'fem', 'm', 'female'], dtype=object)

In [33]:
demographics.groupby('Gender')['Client ID'].nunique()

Gender
0         4906
1         5085
M            3
Male         1
f            1
fem          2
female       1
m            1
Name: Client ID, dtype: int64

In [34]:
#Replcaing the vlaues into 0s and 1s
#Assigned 0 to females and 1 to males
demographics["Gender"]= demographics["Gender"].replace(['f','fem','female'], '0')
demographics["Gender"]= demographics["Gender"].replace(['m','Male','M'], '1')

In [35]:
demographics["Gender"]. unique()

array(['1', '0'], dtype=object)

In [36]:
#Checking for number of females and men
demographics.groupby('Gender')['Client ID'].nunique()

Gender
0    4910
1    5090
Name: Client ID, dtype: int64

In [37]:
#Checking for unique values in the county column
demographics["County"]. unique()

array(['Cork', 'Cavan', 'Dublin', 'Louth', 'Kilkenny', 'Kildare',
       'Carlow', 'Offaly', 'Galway', 'Wicklow', 'Dublin 4', 'Kerry',
       'Waterford', 'Monaghan', 'Mayo', 'Donegal', 'Laois', 'Tipperary',
       'Clare', 'Meath', 'DÛblin', 'Leitrim', 'Longford', 'Limerick',
       'Navan', 'Borris', 'Sandyford', 'Maynooth', 'Portlaoise', 'Trim',
       'Ballina', 'Spain', 'Sligo Town', 'Cork City', 'Rosslare',
       'Northern Ireland', 'Lahinch', 'Adare', 'Dublin 8', 'Dublin 3',
       'Dublin 7', 'Dublin 1', 'DubÝn', 'Boyle', 'Co Cork', 'Co Dublin',
       'Co. Kerry', 'CountyCork', 'Co. Dublin', 'County Cork', '999', ' ',
       'County Mayo', 'Co.. Dublin', 'kildare Town', 'clare', 'dublin',
       'leitrim', 'offaly', 'Kildare town', 'Dublin 30', 'Dublin 21',
       'County Longford', 'Dublin -7', 'Dublin City', 'County - Longford',
       nan], dtype=object)

In [38]:
#There are 67 different values in the County column- many are duplicate 
len(demographics["County"]. unique())

67

In [39]:
#Removing other Special charecters and numbers from the county string
#Converting the string to lower case
demographics["County"] = demographics["County"].str.extract(r'(^[a-zA-Z ]*$)')
demographics["County"] = demographics["County"].str.lower()

In [40]:
#Now the unique county values reduced to 47
len(demographics["County"]. unique())

47

In [41]:
demographics["County"]. unique()

array(['cork', 'cavan', 'dublin', 'louth', 'kilkenny', 'kildare',
       'carlow', 'offaly', 'galway', 'wicklow', nan, 'kerry', 'waterford',
       'monaghan', 'mayo', 'donegal', 'laois', 'tipperary', 'clare',
       'meath', 'leitrim', 'longford', 'limerick', 'navan', 'borris',
       'sandyford', 'maynooth', 'portlaoise', 'trim', 'ballina', 'spain',
       'sligo town', 'cork city', 'rosslare', 'northern ireland',
       'lahinch', 'adare', 'boyle', 'co cork', 'co dublin', 'countycork',
       'county cork', ' ', 'county mayo', 'kildare town',
       'county longford', 'dublin city'], dtype=object)

In [42]:
#demographics.County[demographics['County'].str.match('^.*cork.*$')== True]

In [43]:
#Replcaing the similar counties with standard county names
demographics["County"]= demographics["County"].replace(['dublin city','co dublin'], 'dublin')
demographics["County"]= demographics["County"].replace(['co cork','cork city','county cork','countycork'], 'cork')

In [44]:
#Replacing the empty string county names with None string
demographics["County"]= demographics["County"].replace([' ','nan'],np.nan)

In [45]:
demographics["County"]. unique()

array(['cork', 'cavan', 'dublin', 'louth', 'kilkenny', 'kildare',
       'carlow', 'offaly', 'galway', 'wicklow', nan, 'kerry', 'waterford',
       'monaghan', 'mayo', 'donegal', 'laois', 'tipperary', 'clare',
       'meath', 'leitrim', 'longford', 'limerick', 'navan', 'borris',
       'sandyford', 'maynooth', 'portlaoise', 'trim', 'ballina', 'spain',
       'sligo town', 'rosslare', 'northern ireland', 'lahinch', 'adare',
       'boyle', 'county mayo', 'kildare town', 'county longford'],
      dtype=object)

In [46]:
#Checking for unique Income groups
demographics["Income Group"]. unique()

array(['10001 - 40000', '0 - 10000', '40001 - 60000', '60001 - 100000',
       '100000+', '0  ? 10000', '10002 - 40000', '10001 ?. 40000',
       '10001 ? 40000'], dtype=object)

In [47]:
#Replcaing the special charecters with standard hiphen and replacing 10002 to 10001
demographics["Income Group"]= demographics["Income Group"].str.replace('?', "-")
demographics["Income Group"]= demographics["Income Group"].str.replace('.', "")
demographics["Income Group"]= demographics["Income Group"].str.replace(' ', "")
demographics["Income Group"]= demographics["Income Group"].str.replace('10002', "10001")

In [48]:
#Now we have only 5 income groups
demographics["Income Group"]. unique()

array(['10001-40000', '0-10000', '40001-60000', '60001-100000', '100000+'],
      dtype=object)

In [49]:
#Number of clients per each income group
demographics.groupby("Income Group")['Client ID'].nunique()

Income Group
0-10000          371
100000+         1134
10001-40000     5584
40001-60000     1175
60001-100000    1736
Name: Client ID, dtype: int64

In [50]:
#Setting the age column with a limit, so that improper values are restricted.
maxAge = 100
demographics['Age'].where(demographics['Age'] <= maxAge, maxAge , inplace = True)

In [51]:
demographics.head()

,Client ID,Age,Gender,County,Income Group
0,1,36,1,cork,10001-40000
1,2,43,1,cavan,0-10000
2,3,32,0,dublin,10001-40000
3,4,52,1,louth,40001-60000
4,5,63,0,kilkenny,60001-100000


In [52]:
#Checking for null values
demographics.isnull().sum()

Client ID        0
Age              0
Gender           0
County          28
Income Group     0
dtype: int64

In [53]:
demographics.describe()

,Client ID,Age
count,10000.00000,10000.000000
mean,5000.50000,44.347000
std,2886.89568,14.849857
min,1.00000,20.000000
25%,2500.75000,32.000000
50%,5000.50000,44.000000
75%,7500.25000,57.000000
max,10000.00000,100.000000


<h3>Reading Previous product held file </h3>

In [54]:
pre_product = pd.read_csv("ModelBuild_Product_Held_in_Bank.csv",encoding='latin1', header = None)

In [55]:
pre_product.head()

,0,1
0,1,4
1,2,4
2,3,2
3,4,2
4,5,1


In [56]:
#Naming the columns
pre_product.rename(columns={0:'Client ID', 1: 'No_of_products_held'}, inplace= True)

In [57]:
pre_product.head()

,Client ID,No_of_products_held
0,1,4
1,2,4
2,3,2
3,4,2
4,5,1


In [58]:
pre_product.shape

(10004, 2)

In [59]:
#Checking for the duplicates
pre_product[pre_product["Client ID"].duplicated()]

,Client ID,No_of_products_held
46,46,2
1221,1220,4
3677,3674,2
3678,3675,5


In [60]:
#index of the duplicate rows
i = pre_product[pre_product["Client ID"].duplicated()].index

In [61]:
#removing the duplicate rows
pre_product.drop(i, inplace = True)

In [62]:
pre_product.shape

(10000, 2)

In [63]:
pre_product.dtypes

Client ID               int64
No_of_products_held    object
dtype: object

In [64]:
#Checking for the unique values
pre_product.No_of_products_held.unique()

array(['4', '2', '1', '5', '3', '-1', '-2', ' '], dtype=object)

In [65]:
#Checking for the null values
pre_product.isna().sum()

Client ID              0
No_of_products_held    0
dtype: int64

In [66]:
#Replcaing the empy string with 0
pre_product["No_of_products_held"]= pre_product["No_of_products_held"].str.replace(' ', '0')

In [67]:
#Converting the column to integer values
pre_product['No_of_products_held'] = pre_product['No_of_products_held'].astype(int)

In [68]:
pre_product.No_of_products_held.unique()

array([ 4,  2,  1,  5,  3, -1, -2,  0])

In [69]:
#Converting the column to absolute values and negetive values is possible for this column
pre_product.No_of_products_held = pre_product.No_of_products_held.abs()

In [70]:
pre_product.dtypes

Client ID              int64
No_of_products_held    int64
dtype: object

In [71]:
pre_product.describe()

,Client ID,No_of_products_held
count,10000.00000,10000.000000
mean,5000.50000,3.002700
std,2886.89568,1.410068
min,1.00000,0.000000
25%,2500.75000,2.000000
50%,5000.50000,3.000000
75%,7500.25000,4.000000
max,10000.00000,5.000000


<h3>Reading the Trasactions Amount file</h3>

In [72]:
txn_amount = pd.read_csv("ModelBuild_TXN_Amount.csv",encoding='cp1252',
                  names=['Client ID', 'Average_CA_transaction_amount'],
                  header = None)

In [73]:
#for reading amount use different encoding

In [74]:
txn_amount.head()

,Client ID,Average_CA_transaction_amount
0,1,€58
1,2,"€2,663"
2,3,€46
3,4,€0
4,5,€126


In [75]:
txn_amount.dtypes

Client ID                         int64
Average_CA_transaction_amount    object
dtype: object

In [76]:
txn_amount.shape

(10004, 2)

In [77]:
#Checking for duplicates
txn_amount[txn_amount["Client ID"].duplicated()]

,Client ID,Average_CA_transaction_amount
46,46,€0
1221,1220,€57
3677,3674,€0
3678,3675,"€1,636"


In [78]:
#Finding the index of the duplicates
i = txn_amount[txn_amount["Client ID"].duplicated()].index

In [79]:
#Removing the duplicates
txn_amount.drop(i, inplace = True)

In [80]:
txn_amount.shape

(10000, 2)

In [81]:
txn_amount.describe()

,Client ID
count,10000.00000
mean,5000.50000
std,2886.89568
min,1.00000
25%,2500.75000
50%,5000.50000
75%,7500.25000
max,10000.00000


In [82]:
#sample view of the Transaction amount column
txn_amount.Average_CA_transaction_amount[0]

'€58'

In [83]:
txn_amount.Average_CA_transaction_amount[1]

'€2,663'

In [84]:
#Remvoing the Euro symbol from the column and comma
txn_amount['Average_CA_transaction_amount']= txn_amount.Average_CA_transaction_amount.apply(lambda x: x.strip('€'))
txn_amount['Average_CA_transaction_amount'] = txn_amount['Average_CA_transaction_amount'].str.replace(',', '')

In [85]:
#Convert the column from object to integer
txn_amount['Average_CA_transaction_amount'] = txn_amount['Average_CA_transaction_amount'].astype(int)

In [86]:
txn_amount.head()

,Client ID,Average_CA_transaction_amount
0,1,58
1,2,2663
2,3,46
3,4,0
4,5,126


In [87]:
txn_amount.dtypes

Client ID                        int64
Average_CA_transaction_amount    int64
dtype: object

In [88]:
#Checking for null values
txn_amount.isnull().sum()

Client ID                        0
Average_CA_transaction_amount    0
dtype: int64

In [89]:
txn_amount.describe()

,Client ID,Average_CA_transaction_amount
count,10000.00000,10000.000000
mean,5000.50000,444.829200
std,2886.89568,877.434779
min,1.00000,0.000000
25%,2500.75000,0.000000
50%,5000.50000,30.000000
75%,7500.25000,402.000000
max,10000.00000,9470.000000


<h3>Reading the Transactions file</h3>

In [90]:
transactions = pd.read_csv("TEST_Transactions_out_of_Current_Account.csv",encoding='latin1')

In [91]:
transactions.head()

,Client,Num Transactions,Last TXN Amount,Merchant Code,Last Transaction Narrative,Unnamed: 5,Unnamed: 6,Unnamed: 7,0
0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,17.0,83.66,7211.0,THE BRIDGE LAUNDRY WICKLOW TOWN,NaN,NaN,NaN,NaN
2,3.0,25.0,526.18,3667.0,LUXOR HOTEL/CASINO LAS VEGAS NV,NaN,NaN,NaN,NaN
3,4.0,13.0,70.68,5712.0,HARVEY NORMAN CARRICKMINES,NaN,NaN,NaN,NaN
4,5.0,39.0,259.07,5999.0,PAYPAL *PETEWOODWAR 35314369001,NaN,NaN,NaN,NaN


In [92]:
#Removing the unwanted columns
transactions.drop(['Unnamed: 5','Unnamed: 6','Unnamed: 7','0'], axis = 1, inplace=True)

In [93]:
#Checkinmg for null valued rows
transactions.isnull().sum()

Client                        1217
Num Transactions              1217
Last TXN Amount               4497
Merchant Code                 4497
Last Transaction Narrative    4497
dtype: int64

In [94]:
transactions.dtypes

Client                        float64
Num Transactions              float64
Last TXN Amount               float64
Merchant Code                 float64
Last Transaction Narrative     object
dtype: object

In [95]:
transactions.shape

(11221, 5)

In [96]:
#Checking for duplicates
transactions[transactions["Client"].duplicated()]

,Client,Num Transactions,Last TXN Amount,Merchant Code,Last Transaction Narrative
46,46.0,0.0,NaN,NaN,NaN
1221,1220.0,14.0,492.88,7311.0,THE IRISH TIMES LTD DUBLIN
3676,3674.0,0.0,NaN,NaN,NaN
3678,3675.0,17.0,509.55,8049.0,FOOT STOP LIMITED NAAS
10005,NaN,NaN,NaN,NaN,NaN
10006,NaN,NaN,NaN,NaN,NaN
10007,NaN,NaN,NaN,NaN,NaN
10008,NaN,NaN,NaN,NaN,NaN
10009,NaN,NaN,NaN,NaN,NaN
10010,NaN,NaN,NaN,NaN,NaN


In [97]:
#Index of the 1220 duplicate values
i = transactions[transactions["Client"].duplicated()].index

In [98]:
#Removing the duplicate rows
transactions.drop(i,inplace=True)

In [99]:
transactions.shape

(10001, 5)

In [100]:
#Removing the null value in the client column
transactions = transactions.dropna(axis=0, subset=['Client'])

In [101]:
#Checking the null vlaues
transactions.isnull().sum()

Client                           0
Num Transactions                 0
Last TXN Amount               3278
Merchant Code                 3278
Last Transaction Narrative    3278
dtype: int64

In [102]:
transactions.shape

(10000, 5)

In [103]:
transactions.dtypes

Client                        float64
Num Transactions              float64
Last TXN Amount               float64
Merchant Code                 float64
Last Transaction Narrative     object
dtype: object

In [104]:
#Converting the client column to integer
transactions.Client = transactions.Client.astype(int)

In [105]:
transactions.dtypes

Client                          int64
Num Transactions              float64
Last TXN Amount               float64
Merchant Code                 float64
Last Transaction Narrative     object
dtype: object

In [106]:
#Renaming the column name
transactions.rename(columns = {"Client":"Client ID"}, inplace = True)

In [107]:
transactions.describe()

,Client ID,Num Transactions,Last TXN Amount,Merchant Code
count,10000.00000,10000.000000,6722.000000,6722.000000
mean,5000.50000,18.292900,451.707115,4816.615293
std,2886.89568,22.962819,258.342680,1819.826918
min,1.00000,0.000000,5.390000,742.000000
25%,2500.75000,0.000000,225.352500,3508.000000
50%,5000.50000,10.000000,450.065000,3793.000000
75%,7500.25000,30.000000,674.917500,5950.000000
max,10000.00000,100.000000,899.860000,9950.000000


In [108]:
#Viewing the text column
transactions["Last Transaction Narrative"].head()

0                                         NaN
1       THE BRIDGE LAUNDRY       WICKLOW TOWN
2    LUXOR HOTEL/CASINO       LAS VEGAS    NV
3       HARVEY NORMAN            CARRICKMINES
4        PAYPAL *PETEWOODWAR      35314369001
Name: Last Transaction Narrative, dtype: object

<h3>Reading purchased loan flag file</h3>

In [109]:
purchased_loan = pd.read_csv("TargetVariable_Purchased_Loan_Flag.csv",encoding='latin1')

In [110]:
purchased_loan.head()

,Client ID,Loan Flag,Unnamed: 2,Unnamed: 3
0,1,0,NaN,NaN
1,2,0,NaN,NaN
2,3,0,NaN,NaN
3,4,0,NaN,NaN
4,5,0,NaN,NaN


In [111]:
#Removing the unwanted columns
purchased_loan.drop(['Unnamed: 2','Unnamed: 3'], axis = 1, inplace=True)

In [112]:
purchased_loan.head()

,Client ID,Loan Flag
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [113]:
purchased_loan.dtypes

Client ID    int64
Loan Flag    int64
dtype: object

In [114]:
purchased_loan.shape

(10004, 2)

In [115]:
#Checking for duplicates
purchased_loan[purchased_loan["Client ID"].duplicated()]

,Client ID,Loan Flag
46,46,0
1221,1220,0
3677,3674,0
3678,3675,0


In [116]:
#Index of the duplicate rows
i = purchased_loan[purchased_loan["Client ID"].duplicated()].index

In [117]:
#Removing the duplicate rows
purchased_loan.drop(i,inplace = True)

In [118]:
purchased_loan.shape

(10000, 2)

In [119]:
purchased_loan.describe()

,Client ID,Loan Flag
count,10000.00000,10000.000000
mean,5000.50000,0.021100
std,2886.89568,0.143725
min,1.00000,0.000000
25%,2500.75000,0.000000
50%,5000.50000,0.000000
75%,7500.25000,0.000000
max,10000.00000,1.000000


In [120]:
#Checking for the null values
purchased_loan.isnull().sum()

Client ID    0
Loan Flag    0
dtype: int64

In [121]:
purchased_loan["Loan Flag"].unique()

array([0, 1])

<h2>Merging the Dataframes to form a single dataframe</h2>

In [122]:
#Checking the shapes of all the read dataframes
print(purchased_loan.shape)
print(transactions.shape)
print(txn_amount.shape)
print(demographics.shape)
print(pre_loan.shape)
print(pre_product.shape)

(10000, 2)
(10000, 5)
(10000, 2)
(10000, 5)
(10000, 2)
(10000, 2)


In [123]:
#Merging the demographics and transaction amount dataframes
final_df = pd.merge(demographics,txn_amount,  how='left', left_on=['Client ID'], right_on = ['Client ID'])

In [124]:
final_df.head()

,Client ID,Age,Gender,County,Income Group,Average_CA_transaction_amount
0,1,36,1,cork,10001-40000,58
1,2,43,1,cavan,0-10000,2663
2,3,32,0,dublin,10001-40000,46
3,4,52,1,louth,40001-60000,0
4,5,63,0,kilkenny,60001-100000,126


In [125]:
final_df.shape

(10000, 6)

In [126]:
#Merging the final dataframe and previous products dataframe
final_df = pd.merge(final_df,pre_product,  how='left', left_on=['Client ID'], right_on = ['Client ID'])

In [127]:
final_df.head()

,Client ID,Age,Gender,County,Income Group,Average_CA_transaction_amount,No_of_products_held
0,1,36,1,cork,10001-40000,58,4
1,2,43,1,cavan,0-10000,2663,4
2,3,32,0,dublin,10001-40000,46,2
3,4,52,1,louth,40001-60000,0,2
4,5,63,0,kilkenny,60001-100000,126,1


In [128]:
final_df.shape

(10000, 7)

In [129]:
#Merging the final dataframe and previous Loan dataframe
final_df = pd.merge(final_df,pre_loan,  how='left', left_on=['Client ID'], right_on = ['Client ID'])

In [130]:
final_df.head()

,Client ID,Age,Gender,County,Income Group,Average_CA_transaction_amount,No_of_products_held,Held Loan previously
0,1,36,1,cork,10001-40000,58,4,1
1,2,43,1,cavan,0-10000,2663,4,0
2,3,32,0,dublin,10001-40000,46,2,0
3,4,52,1,louth,40001-60000,0,2,1
4,5,63,0,kilkenny,60001-100000,126,1,0


In [131]:
final_df.shape

(10000, 8)

In [132]:
#Merging the final dataframe and Transactionsdataframe
final_df = pd.merge(final_df,transactions,  how='left', left_on=['Client ID'], right_on = ['Client ID'])

In [133]:
final_df.head()

,Client ID,Age,Gender,County,Income Group,Average_CA_transaction_amount,No_of_products_held,Held Loan previously,Num Transactions,Last TXN Amount,Merchant Code,Last Transaction Narrative
0,1,36,1,cork,10001-40000,58,4,1,0.0,NaN,NaN,NaN
1,2,43,1,cavan,0-10000,2663,4,0,17.0,83.66,7211.0,THE BRIDGE LAUNDRY WICKLOW TOWN
2,3,32,0,dublin,10001-40000,46,2,0,25.0,526.18,3667.0,LUXOR HOTEL/CASINO LAS VEGAS NV
3,4,52,1,louth,40001-60000,0,2,1,13.0,70.68,5712.0,HARVEY NORMAN CARRICKMINES
4,5,63,0,kilkenny,60001-100000,126,1,0,39.0,259.07,5999.0,PAYPAL *PETEWOODWAR 35314369001


In [134]:
final_df.shape

(10000, 12)

In [135]:
#Merging the final dataframe and purchased loan dataframe
final_df = pd.merge(final_df,purchased_loan,  how='left', left_on=['Client ID'], right_on = ['Client ID'])

In [136]:
final_df.head()

,Client ID,Age,Gender,County,Income Group,Average_CA_transaction_amount,No_of_products_held,Held Loan previously,Num Transactions,Last TXN Amount,Merchant Code,Last Transaction Narrative,Loan Flag
0,1,36,1,cork,10001-40000,58,4,1,0.0,NaN,NaN,NaN,0
1,2,43,1,cavan,0-10000,2663,4,0,17.0,83.66,7211.0,THE BRIDGE LAUNDRY WICKLOW TOWN,0
2,3,32,0,dublin,10001-40000,46,2,0,25.0,526.18,3667.0,LUXOR HOTEL/CASINO LAS VEGAS NV,0
3,4,52,1,louth,40001-60000,0,2,1,13.0,70.68,5712.0,HARVEY NORMAN CARRICKMINES,0
4,5,63,0,kilkenny,60001-100000,126,1,0,39.0,259.07,5999.0,PAYPAL *PETEWOODWAR 35314369001,0


In [137]:
final_df.shape

(10000, 13)

In [138]:
#Saving the merged dataset
final_df.to_csv("mergedData.csv", index = False)